# Neural Network
make a neural network that learns to distinguish between an even number and an odd number
predict odd.


#### stochastic gradient descent (SGD)

A gradient descent algorithm in which the batch size is one. In other words, SGD relies on a single example chosen uniformly at random from a dataset to calculate an estimate of the gradient at each step.


#### gradient descent

A technique to minimize loss by computing the gradients of loss with respect to the model's parameters, conditioned on training data. Informally, gradient descent iteratively adjusts parameters, gradually finding the best combination of weights and bias to minimize loss.

#### loss

A measure of how far a model's predictions are from its label. Or, to phrase it more pessimistically, a measure of how bad the model is. To determine this value, a model must define a loss function. For example, linear regression models typically use mean squared error for a loss function, while logistic regression models use Log Loss.

#### prediction

A model's output when provided with an input example.

#### example

One row of a dataset.

In [226]:
import math
import random

def sigmoid(z):
    return 1.0/(1.0+math.exp(-z))

def logloss(true_label, predicted_prob):
    if true_label == 1:
        return -log(predicted_prob)
    else:
        return -log(1 - predicted_prob)

In [227]:
class inputNeuron:
    def __init__(self,activation):
        '''
        inputNeuron contains only an activation value as given, no bias or weights.
        
        input:
        activation: numerical value from 0 to 1
        
        attri:
        activation: numerical value from 0 to 1
        '''
        self.activation = activation

            
class neuron:
    def __init__(self,prevActs):
        '''
        neuron contains randomized weights and bias, 
        and an activation value calculated from said weights and bias
        
        input:
        prevActs: list of activation values from previous layer
        
        attri:
        weights: list of randomized number of weight associated with a node in previous layer
        bias: randomized number 
        activation: calculated from applying the sigmoid function
        
        '''
        
        self.bias = random.random()  #random
        sig=(-self.bias)
        
        self.weights = []
        for i in range(len(prevActs)):
            
            currweight=random.random() #random
            
            self.weights.append(currweight)
            sig+=prevActs[i] * currweight
            
        self.activation = sigmoid(sig)
    
    #def changeto(self, weights, bias)
       

class layer:
    def __init__(self, prevActs,numNeuron, inputLayer = False):
        '''
        a layer is made up of "numNeuron" number of neurons.
        
        input:
        numNneuron: number of neurons in a layer
        prevActs: list of activations in previous layer
        
        attri:
        neurons: list of all neurons at current layer
        acts: list of activations of those neurons at current layer
        '''
        self.neurons = []
        self.acts = []
        if inputLayer == False:
            for i in range(numNeuron):
                currNeuron = neuron(prevActs)
                self.neurons.append(currNeuron)
                self.acts.append(currNeuron.activation)
        else:
            for i in range(numNeuron):
                currNeuron = inputNeuron(random.random()) #random
                self.neurons.append(currNeuron)
                self.acts.append(currNeuron.activation)
        
class network:
    def __init__(self, neuronNums):
        '''
        a network contains an inputLayer, a number of hidden layers and an output layer.
        
        input:
        neuronNums: a list of number of neurons in each consecutive layer of the network
        
        attributes:
        layers: a list of layers in the network.     
        '''
        self.layers = []
        
        #input layer
        layer1st = layer([],neuronNums[0], inputLayer = True)
        self.layers.append(layer1st)
        prevActs = layer1st.acts
        
        #hidden layer(s) + output layer
        for numNeuron in neuronNums[1:]:
            currLayer = layer(prevActs,numNeuron)
            self.layers.append(currLayer)
            prevActs = currLayer.acts
    
    def forward(self, inputActs):
        '''
        inputActs: list of activation(s) for input layer
        '''
        
        for i in range(len(inputActs)):
            self.layers[0].neurons[i].activation = inputActs[i]
        prevActs = inputActs
        
        for nLayer in range(1,len(self.layers)): #for each non-input layer in network
            for nNeuron in range(len(self.layers[nLayer].neurons)): #for each neuron in layer
                currNeuron = self.layers[nLayer].neurons[nNeuron]
                sig=(-currNeuron.bias)
                
                for w in range(len(prevActs)): #for each weight connection in neuron
                    sig += prevActs[w] * currNeuron.weights[w]
                    
                currNeuron.activation = sigmoid(sig)
                currNeuron = self.layers[nLayer].acts[nNeuron] = currNeuron.activation
                
            prevActs = self.layers[nLayer].acts
    



In [228]:
mynetwork = network([1,8,1])

In [229]:
mynetwork.layers[0].neurons[0].activation

0.7850482634786273

In [230]:
mynetwork.layers[-1].neurons[0].activation

0.8714564450578488

In [231]:
mynetwork.forward([6])

In [232]:
mynetwork.layers[0].neurons[0].activation

6

In [233]:
mynetwork.layers[-1].neurons[0].activation

0.9783641662666129

#### Make training and testing dataset:


In [234]:
def makedata(datapointNum):
    """
    Returns "dataset": a list of examples.
    Each example is a tuple.
    Each tuple contains (number,label).
    number ranges between 0 and 100001
    label 0 is even, 1 is odd.
    """
    dataset = []
    for i in range(datapointNum):
        label=random.randint(0,1)
        if label==0:
            #generate even number
            even = 2*random.randint(0,50000)
            dataset.append((even,label))
            
        else:
            #generate odd number
            odd = 2*(random.randint(1,50000)//2)+1
            dataset.append((odd,label))
    return dataset

In [235]:
makedata(5)

[(57976, 0), (62972, 0), (83850, 0), (17997, 1), (11644, 0)]

In [236]:
train = makedata(3)
test = makedata(1000)

Train the network:
- calculate the cost/loss for each traning datapoint and take their average for cost of the network.
- Find gradient descend through back propagation

In [237]:
def loss(network,data):
    loss = 0
    for dtpt in data:
        network.forward([dtpt[0]])
        predicted_prob = mynetwork.layers[-1].neurons[0].activation
        true_label = dtpt[1]
        loss += logloss(true_label, predicted_prob)
    return loss